In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("flights_2022_sample_1_5m.csv")

In [3]:
#Defining Target Column
df['DELAYED'] = np.where(df['ARR_DELAY'] >= 15,1,0).astype(int)

In [4]:
#   Dropping Data Leaking Columns

leakage_cols = [
    "ARR_DELAY",
    "ARR_TIME",
    "WHEELS_OFF",
    "WHEELS_ON",
    "TAXI_IN",
    "TAXI_OUT"
]

df.drop(columns=[c for c in leakage_cols if c in df.columns],inplace=True)

In [5]:
#   Extracting 'Months','Days','Weekend' from Flight date
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'],errors='coerce')
df['MONTH'] = df['FL_DATE'].dt.month
df['DAY_OF_WEEK'] = df['FL_DATE'].dt.dayofweek
df['IS_WEEKEND'] = df['DAY_OF_WEEK'].isin([5,6]).astype(int)

df.drop(columns=['FL_DATE'],inplace=True)

In [6]:
#   Extracting Departure Hour

df['DEP_TIME_CLEAN'] = (df['DEP_TIME'].fillna(0).astype(str).str.replace('.0','',regex=False).str.zfill(4))
df['DEP_HOUR'] = df['DEP_TIME_CLEAN'].str[:2].astype(int)

df.loc[df['DEP_HOUR']>= 24,'DEP_HOUR'] = np.nan

df.drop(columns=['DEP_TIME_CLEAN','DEP_TIME'],inplace=True)

In [7]:
delay_cols = [
    'DEP_DELAY',
    'AIR_TIME',
    'CRS_ELAPSED_TIME'
]
for col in delay_cols:
    if col in df.columns:
        df[col+'_MISSING'] = df[col].isna().astype(int)
        df[col] = df[col].fillna(0)

In [8]:
time_cols = ['DEP_HOUR']

for col in time_cols:
    if col in df.columns:
        df[col+'_MISSING'] = df[col].isna().astype(int)
        df[col].fillna(df[col].median())

In [9]:
if 'DISTANCE' in df.columns:
    df['DISTANCE'] = df['DISTANCE'].fillna(df['DISTANCE'].median())

In [10]:
cat_cols = df.select_dtypes(exclude='number').columns
df[cat_cols] = df[cat_cols].fillna('UNKNOWN')

In [11]:
df.isnull().sum().sort_values(ascending=False)

DELAY_DUE_LATE_AIRCRAFT     1232295
DELAY_DUE_SECURITY          1232295
DELAY_DUE_NAS               1232295
DELAY_DUE_WEATHER           1232295
DELAY_DUE_CARRIER           1232295
ELAPSED_TIME                  43133
DEP_HOUR                        120
AIR_TIME                          0
CRS_ELAPSED_TIME_MISSING          0
AIR_TIME_MISSING                  0
DEP_DELAY_MISSING                 0
IS_WEEKEND                        0
DAY_OF_WEEK                       0
MONTH                             0
DELAYED                           0
DISTANCE                          0
AIRLINE                           0
AIRLINE_DOT                       0
CRS_ELAPSED_TIME                  0
DIVERTED                          0
CANCELLATION_CODE                 0
CANCELLED                         0
CRS_ARR_TIME                      0
DEP_DELAY                         0
CRS_DEP_TIME                      0
DEST_CITY                         0
DEST                              0
ORIGIN_CITY                 

In [12]:
def frequency_encode(series):
    freq = series.value_counts()/len(series)
    return series.map(freq)

for col in ['OP_CARRIER','ORIGIN','DEST']:
    if col in df.columns:
        df[col + '_FREQ'] = frequency_encode(df[col])
        df.drop(columns=col,inplace=True)

In [13]:
leaky_cols = [
    'DELAY_DUE_CARRIER',
    'DELAY_DUE_WEATHER',
    'DELAY_DUE_NAS',
    'DELAY_DUE_SECURITY',
    'DELAY_DUE_LATE_AIRCRAFT',
    'DEP_DELAY',
    'AIR_TIME',
    'ELAPSED_TIME',
    'CANCELLED',
    'DIVERTED',
    'CANCELLATION_CODE',
    'DEP_HOUR_MISSING',
    'DEP_DELAY_MISSING'
]
df.drop(columns=[c for c in leaky_cols if c in df.columns],inplace=True)

In [14]:
df.columns

Index(['AIRLINE', 'AIRLINE_DOT', 'AIRLINE_CODE', 'DOT_CODE', 'FL_NUMBER',
       'ORIGIN_CITY', 'DEST_CITY', 'CRS_DEP_TIME', 'CRS_ARR_TIME',
       'CRS_ELAPSED_TIME', 'DISTANCE', 'DELAYED', 'MONTH', 'DAY_OF_WEEK',
       'IS_WEEKEND', 'DEP_HOUR', 'AIR_TIME_MISSING',
       'CRS_ELAPSED_TIME_MISSING', 'ORIGIN_FREQ', 'DEST_FREQ'],
      dtype='object')

In [15]:
#   Frequency Encoding for categorical columns

catego_cols = [
    'AIRLINE',
    'AIRLINE_DOT',
    'AIRLINE_CODE',
    'ORIGIN_CITY',
    'DEST_CITY'
]

for col in catego_cols:
    freq = df[col].value_counts(normalize=True)
    df[col + '_FREQ'] = df[col].map(freq)


In [16]:
df.drop(columns=[c for c in catego_cols if c in df.columns],inplace=True)

In [17]:
df.isnull().sum()

DOT_CODE                      0
FL_NUMBER                     0
CRS_DEP_TIME                  0
CRS_ARR_TIME                  0
CRS_ELAPSED_TIME              0
DISTANCE                      0
DELAYED                       0
MONTH                         0
DAY_OF_WEEK                   0
IS_WEEKEND                    0
DEP_HOUR                    120
AIR_TIME_MISSING              0
CRS_ELAPSED_TIME_MISSING      0
ORIGIN_FREQ                   0
DEST_FREQ                     0
AIRLINE_FREQ                  0
AIRLINE_DOT_FREQ              0
AIRLINE_CODE_FREQ             0
ORIGIN_CITY_FREQ              0
DEST_CITY_FREQ                0
dtype: int64

In [18]:
# Filling 120 missing DEP_HOUR with median 
 
median_dep_hour = df['DEP_HOUR'].median()
df['DEP_HOUR'] = df['DEP_HOUR'].fillna(median_dep_hour)


In [19]:
#   final check

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 20 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   DOT_CODE                  1500000 non-null  int64  
 1   FL_NUMBER                 1500000 non-null  int64  
 2   CRS_DEP_TIME              1500000 non-null  int64  
 3   CRS_ARR_TIME              1500000 non-null  int64  
 4   CRS_ELAPSED_TIME          1500000 non-null  float64
 5   DISTANCE                  1500000 non-null  float64
 6   DELAYED                   1500000 non-null  int32  
 7   MONTH                     1500000 non-null  int32  
 8   DAY_OF_WEEK               1500000 non-null  int32  
 9   IS_WEEKEND                1500000 non-null  int32  
 10  DEP_HOUR                  1500000 non-null  float64
 11  AIR_TIME_MISSING          1500000 non-null  int32  
 12  CRS_ELAPSED_TIME_MISSING  1500000 non-null  int32  
 13  ORIGIN_FREQ               1

In [20]:
df.to_csv(
    "ready_for_modeling.csv", index=False
)